In [1]:
from pathlib import Path
from tqdm import tqdm
import json
import pysubs2

videoRoot = Path(r'\\acepc-gk3\Seagate 8T\categorized_mp4')

In [2]:
allMrkermapFiles = [p for p in tqdm(videoRoot.glob('**/*.markermap')) if p.with_suffix('.ass.original').is_file()]

4085it [00:31, 131.07it/s]


In [3]:
fileList = []
for markermapPath in tqdm(allMrkermapFiles):
    with markermapPath.open() as f:
        markermap = json.load(f)
        for k,v in markermap.items():
            if '_groundtruth' in v and 'subtitles' in v and  v['_groundtruth'] == 0.0 and v['subtitles'] == 1.0:
                fileList.append(markermapPath)
                break        

100%|██████████████████████████████████████████████████████████████████████| 1675/1675 [00:26<00:00, 62.46it/s]


In [5]:
import re
def ExtractSubtitlesText(assPath: Path, clip: tuple[float, float]) -> str:
    subs = pysubs2.load(assPath)
    result = []
    for event in subs:
        start, end = clip[0] * 1000, clip[1] * 1000
        # if overlap
        if event.start < end and start < event.end:
            text = event.text
            text = re.sub(r'\{.*?\}', '', text)
            text = text.replace(r'\N', '')
            result.append(text)
    return ' '.join(result)

ExtractSubtitlesText(
    r"\\acepc-gk3\Seagate 8T\categorized_mp4\ドラマ\シッコウ!!~犬と私と執行官~\_metadata\2023年07月11日21時00分00秒-シッコウ！！～犬と私と執行官～　＃２[解][字].ass.original",
    (5, 40))


'とどろき はねと (轟木羽人)街も眠っております｡ どうも ハットくんです｡ ピッピピー！ (轟木)という事で 今日のハットの びっくりッピーはですね→ 前回の動画 見た？ 相方のブケショがさ…｡ ふみつぐ (山田史嗣)すいません｡ 僕 もう 動画やめます｡ おい ブケショ…｡ おい ブケショ！ で もう 腹立ったんで 今から押しかけて→ やつ 奴を 丸ごと 僕のテーマカラー ピンクに塗りたくってしまう→ そういう びっくりッピーです｡ ここが ブケショが住んでる マンションです｡'

In [7]:
cmTextList = []
nonCmTextList = []

for markermapPath in tqdm(fileList):
    with markermapPath.open() as f:
        markermap = json.load(f)
        assPath = markermapPath.with_suffix('.ass.original')
        for k,v in markermap.items():
            clip = eval(k)
            # skip very short clips
            if clip[1] - clip[0] < 5:
                continue
            # only take first 15 seconds
            if clip[1] - clip[0] > 15:
                clip = (clip[0], clip[0] + 15)
            text = ExtractSubtitlesText(assPath, clip)
            if v['_groundtruth'] == 0.0 and v['subtitles'] == 1.0:
                cmTextList.append(text)
            elif v['_groundtruth'] == 1.0 and v['subtitles'] == 1.0:
                nonCmTextList.append(text)
len(cmTextList), len(nonCmTextList)

100%|████████████████████████████████████████████████████████████████████████| 755/755 [16:15<00:00,  1.29s/it]


(4131, 5556)

In [8]:
from statistics import stdev

#stdev(cmLen), stdev(nonCmLen)

In [9]:
cmTextList[:20]

['＜日やけによるシミ予防＞ 始めよ？ ＜トーンアップ 出た＞',
 'たなか (田中)まだ始めてないの？ (友人)えっ？ 日やけによる シミを➡ 防ぐ！ ｢ニベア｣のＵＶでね｡ ＜｢ニベア｣最強 予防美容処方＞ ＜日やけによるシミ予防＞ 始めよ？ ＜トーンアップ 出た＞',
 'たけうちりょうま いがい (竹内涼真)ﾎﾞｸは どこでもﾄﾞｱ以外に ないでしょ｡ あらき ゆうこ てっぱん (新木優子)鉄板の｡ すぐ… い じかん はい 入り時間 すぐ入れちゃうから｡ しごと つか 仕事で使うんだ？ うん｡ せーの…｡',
 'なかむら (中村アン)ドラえもん や た どら焼きを いっぱい食べて→ けんこう 健康でいてください｡ たまもりゆうた (玉森裕太) ドラえもん！ これからも→ た なかよ のび太くんと仲良くしてください｡ ふたり (２人)せーの…｡',
 'えいじ (ウエンツ瑛士)ボクは の タイムマシンに乗りたいですね｡ おおた ひかり (太田 光)ﾀｲﾑﾏｼﾝはね そりゃあ…｡ (ｳｴﾝﾂ)乗りたいです｡ たなか ゆうじ (田中裕二)行きたいとこあんの？ みらい かこ 未来とか過去とか｡ ボクは未来ですね｡ ねん 20年ぐらいがいいですね｡ てまえ 60手前ぐらいです ボクでいうと｡',
 'すだ かおう (菅田)花王が ついに突き止めた…｡ かく (賀来)｢菌の隠れ家｣ まみや (間宮)除菌洗剤も漂白剤も ダメだった｡ すぎの ゼロ (杉野)新｢アタックＺＥＲＯ｣なら…｡ (一同)いざ！ まつざか (松坂)バイオクラッシュ洗浄！',
 '≪ルンゴー！ ≪ルンゴー！ ≪ルンゴー！ ≪ワッハッハッハッハー！',
 '(３人)はじめよう ｢ニベアメン｣ さかもと (坂本)なんか 最近 調子いいよね？ ながの (長野)やっぱり？ 肌のハリ？ (２人)おー！ いのはら (井ノ原)続けるって大事！ ＜いい肌と､進み続けろ｡＞ ＜40歳からのスキンケア＞',
 'ツバキ ＜あの ｢TSUBAKI｣から➡ ダメージケアの黒 誕生＞ ＜イオンの力が 髪を根本から補修＞ ＜艶あふれるオーラ髪へ ｢黒TSUBAKI｣＞',
 'にかいどう (二階堂)生理だからこそ できるだけ 気持ちいいものを➡ 身につけたい｡ うん｡ ＜コットン 100％＞ すはだ ♬｢しあわせ素肌｣ ＜

In [10]:
nonCmTextList[:20]

['べにこ ぜにてんどう (紅子)銭天堂｡ ここは 幸運のお客様だけが たどりつける➡ だがしや 駄菓子屋でござんす｡ かみひとえ 運も不運も 紙一重｡',
 '(こはく) うわ～い！ サンマだにゃ～！ うにゃうにゃ うにゃうにゃ…｡ まだまだ いくらでも食べられるにゃ～｡ (こがね)こはく君… うるさくて ねむれませんにゃ…｡ そんな うまいの？',
 'べにこ (紅子)旅行中に出会った わがしや 和菓子屋のはるかさん｡ お兄さんが コンテスト用の 和菓子のアイデアを➡ ライバルに ぬすまれてしまい➡ 優勝を のがしてしまったのでござんす｡',
 '(大室)そんな古くさい菓子 後回しでいい！ おれが考えた 新作の菓子に取りかかってくれ！ つきよねこ でも それでは うちの名物｢月夜猫｣の 材料が足りなくなります｡ そっちは ほかの安い材料で ちゃっちゃと作れ！',
 '♬～ べにこ ぜにてんどう (紅子)銭天堂｡ ここは 幸運のお客様だけが たどりつける➡ だがしや 駄菓子屋でござんす｡ かみひとえ 運も不運も 紙一重｡',
 '♬～ べにこ ぜにてんどう (紅子)銭天堂｡ ここは 幸運のお客様だけが たどりつける➡ だがしや 駄菓子屋でござんす｡ かみひとえ 運も不運も 紙一重｡ ころりと転じる運だめし｡',
 'すっげえ！ リアルなトラ！ こいつは コブラか｡ サイに ライオン➡ ゴリラに オオカミもある｡ 食べるのが もったいないな｡ あっ サイは そんなに好きじゃないから 食べちゃおう｡',
 '♬～ べにこ ぜにてんどう (紅子)銭天堂｡ ここは 幸運のお客様だけが たどりつける➡ だがしや 駄菓子屋でござんす｡ かみひとえ 運も不運も 紙一重｡ ころりと転じる運だめし｡',
 'お客様 心残りが おありでござんすか？ えっ？ ｢逆襲ジンジャーエール｣を 選べばよかったと➡ 少し後かいしてるんじゃござんせんか？ ええ…｡',
 '(ドラえもん) じかん 『ドラえもん』の時間だよ｡ (ドラえもんのいびき) ものおと ≪(物音) …ん？',
 'ディー お ｄボタンを押して→ かお えら しょうぶ ｸﾞｰ ﾁｮｷ ﾊﾟｰの顔を選んで勝負！ あ ﾎﾟｲﾝﾄをためてﾌﾟﾚｾﾞﾝﾄを当てよう！',
 'みんなの｢できたらいいな｣を だいぼしゅう 大募集！ たんじょ

In [ ]:
#!pip install ffmpeg-python
#!pip install SpeechRecognition

In [11]:
import ffmpeg
ffmpegPath = r"C:\Software\ffmpeg\ffmpeg-5.0.1-essentials_build\bin\ffmpeg.exe"

stream = ffmpeg.input(r"C:\Samples\lupin\CM\0096.435-0126.432.ts")
stream = ffmpeg.output(stream, r"C:\Samples\lupin\CM\0096.435-0126.432.wav")

ffmpeg.run(stream, overwrite_output=True, cmd=ffmpegPath)

(None, None)

In [12]:
import speech_recognition as sr
r = sr.Recognizer()

audio_file = r"C:\Samples\lupin\CM\0096.435-0126.432.wav"
with sr.AudioFile(audio_file) as source:
    audio = r.record(source)

text = r.recognize_google(audio, language='ja-JP')

print(text)

ファイブイズホーム 現場に立ってお客様の生活をイメージする それがベストなプレーに繋がっていくんで ご希望通りは当たり前 お客様に感動してもらえるところまで持って行こうと 見えない所こそ 徹底的にえ そんなとこまでっていう 細かい 検査ですね 5 ホームには自信があります 確かな品質 安心価格の
